In [ ]:
# ドライブをマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 自身のColab環境によってディレクトリを変更して下さい
import os
os.chdir("/content/drive/MyDrive/ColabNotebooks/nishika/hate_speech")

In [ ]:
!pip install datasets
!pip install transformers
!pip install sentencepiece==0.1.91

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 14.0 MB/s 
     |████████████████████████████████| 212 kB 68.1 MB/s 
     |████████████████████████████████| 115 kB 71.2 MB/s 
     |████████████████████████████████| 163 kB 72.2 MB/s 
     |████████████████████████████████| 95 kB 6.1 MB/s 
     |████████████████████████████████| 127 kB 65.1 MB/s 
     |████████████████████████████████| 115 kB 75.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Uninstalling dill-0.3.6:
      Successfully uninstalled dill-0.3.6
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 14.6 MB/s 
     |███████████████████████████████

In [ ]:
import os
import datasets
import numpy as np
import pandas as pd
from datasets import load_metric
from transformers import (
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    T5Tokenizer,
    Trainer,
    TrainingArguments,
)

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=77)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

if __name__ == "__main__":

    os.environ["WANDB_DISABLED"] = "true"
    metric = load_metric("f1")

    df = pd.read_csv("/content/drive/MyDrive/input/nishika/hate_speech/train.csv")

    tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-medium")
    tokenizer.pad_token = tokenizer.eos_token

    test = pd.read_csv("/content/drive/MyDrive/input/nishika/hate_speech/test.csv")
    test_dataset = datasets.Dataset.from_pandas(test[["text"]])
    test_tokenized = test_dataset.map(preprocess_function, batched=True)

    train_dataset = datasets.Dataset.from_pandas(df[["text", "label"]])
    train_tokenized = train_dataset.map(preprocess_function, batched=True)

    val_dataset = datasets.Dataset.from_pandas(df[["text", "label"]])
    val_tokenized = val_dataset.map(preprocess_function, batched=True)

    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    model = AutoModelForSequenceClassification.from_pretrained(
        "rinna/japanese-gpt2-medium", num_labels=2
    )
    model.config.pad_token_id = model.config.eos_token_id

    fold_id = 0

    training_args = TrainingArguments(
        output_dir=f"../tmp/results{fold_id}",
        learning_rate=4e-5,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=64,
        num_train_epochs=3,
        weight_decay=0.01,
        evaluation_strategy="steps",
        eval_steps=250,
        load_best_model_at_end=True,
        save_steps=1000,
        gradient_accumulation_steps=3,
        save_total_limit=3,
        fp16=True,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_tokenized,
        eval_dataset=val_tokenized,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    trainer.train()

    oof_results = trainer.predict(test_dataset=val_tokenized)
    np.save(f"oof_prediction{fold_id}", oof_results.predictions)

    results = trainer.predict(test_dataset=test_tokenized)
    np.save(f"test_prediction{fold_id}", results.predictions)

    test["isFake"] = np.argmax(results.predictions, axis=-1)
    test[["id", "isFake"]].to_csv("submission.csv", index=False)

loading file spiece.model from cache at /root/.cache/huggingface/hub/models--rinna--japanese-gpt2-medium/snapshots/f464b76739c884d8b0479a0a7705b7fa71c3fd5a/spiece.model
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--rinna--japanese-gpt2-medium/snapshots/f464b76739c884d8b0479a0a7705b7fa71c3fd5a/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--rinna--japanese-gpt2-medium/snapshots/f464b76739c884d8b0479a0a7705b7fa71c3fd5a/tokenizer_config.json


  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--rinna--japanese-gpt2-medium/snapshots/f464b76739c884d8b0479a0a7705b7fa71c3fd5a/config.json
Model config GPT2Config {
  "_name_or_path": "rinna/japanese-gpt2-medium",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 1,
  "embd_pdrop": 0.1,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1024,
  "n_head": 16,
  "n_inner": 4096,
  "n_layer": 24,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      

Step,Training Loss,Validation Loss,F1
250,No log,0.064694,0.743925
500,0.131800,0.019723,0.948760
750,0.131800,0.011981,0.978441
1000,0.023100,0.003098,0.990260
1250,0.023100,0.000942,0.995106


The following columns in the evaluation set don't have a corresponding argument in `GPT2ForSequenceClassification.forward` and have been ignored: text. If text are not expected by `GPT2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 5256
  Batch size = 64
The following columns in the evaluation set don't have a corresponding argument in `GPT2ForSequenceClassification.forward` and have been ignored: text. If text are not expected by `GPT2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 5256
  Batch size = 64
The following columns in the evaluation set don't have a corresponding argument in `GPT2ForSequenceClassification.forward` and have been ignored: text. If text are not expected by `GPT2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 5256
  Batch size = 64
The following

The following columns in the test set don't have a corresponding argument in `GPT2ForSequenceClassification.forward` and have been ignored: text. If text are not expected by `GPT2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 3223
  Batch size = 64


In [ ]:
import os
import datasets
import numpy as np
import pandas as pd
from datasets import load_metric
from sklearn.model_selection import StratifiedKFold
from transformers import (
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    T5Tokenizer,
    Trainer,
    TrainingArguments,
)


def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=512)


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


if __name__ == "__main__":

    os.environ["WANDB_DISABLED"] = "true"
    metric = load_metric("f1")

    df = pd.read_csv("/content/drive/MyDrive/input/nishika/hate_speech/train.csv")
    df.text = [
        data[: data.find("。")] + "。[SEP]" + data[data.find("。") + 1 :]
        for data in df.text
    ]
    # df.columns = ["id", "label", "text"]

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    for n, (train_index, val_index) in enumerate(cv.split(df, df["label"])):
        df.loc[val_index, "fold"] = int(n)
    df["fold"] = df["fold"].astype(int)

    tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-medium")
    tokenizer.pad_token = tokenizer.eos_token

    test = pd.read_csv("/content/drive/MyDrive/input/nishika/hate_speech/test.csv")
    test.text = [
        data[: data.find("。")] + "。[SEP]" + data[data.find("。") + 1 :]
        for data in test.text
    ]
    test_dataset = datasets.Dataset.from_pandas(test[["text"]])
    test_tokenized = test_dataset.map(preprocess_function, batched=True)

    test_prediction0 = np.load(
        "test_prediction0.npy"
    )
    test["pred0"] = test_prediction0[:, 0]
    test["pred1"] = test_prediction0[:, 1]
    test["label"] = (test["pred0"] < test["pred1"]).astype(int)
    test_filter = test.query("pred0 > 3 or pred0 < -3")[["id", "label", "text"]]

    for fold_id in range(5):

        if fold_id in (0, 1, 2, 3, 4):

            train_dataset = datasets.Dataset.from_pandas(
                pd.concat([df.query(f"fold != {fold_id}"), test_filter]).reset_index(
                    drop=True
                )[["text", "label"]]
            )
            train_tokenized = train_dataset.map(preprocess_function, batched=True)

            val_dataset = datasets.Dataset.from_pandas(
                df.query(f"fold == {fold_id}")[["text", "label"]]
            )
            val_tokenized = val_dataset.map(preprocess_function, batched=True)

            data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

            model = AutoModelForSequenceClassification.from_pretrained(
                "rinna/japanese-gpt2-medium", num_labels=2
            )
            model.config.pad_token_id = model.config.eos_token_id

            training_args = TrainingArguments(
                output_dir=f"../tmp/results{fold_id}",
                learning_rate=4e-5,
                per_device_train_batch_size=4,
                per_device_eval_batch_size=64,
                num_train_epochs=6,
                weight_decay=0.01,
                evaluation_strategy="steps",
                eval_steps=250,
                load_best_model_at_end=True,
                save_steps=1000,
                gradient_accumulation_steps=3,
                save_total_limit=3,
                fp16=True,
            )

            trainer = Trainer(
                model=model,
                args=training_args,
                train_dataset=train_tokenized,
                eval_dataset=val_tokenized,
                tokenizer=tokenizer,
                data_collator=data_collator,
                compute_metrics=compute_metrics,
            )

            trainer.train()

            oof_results = trainer.predict(test_dataset=val_tokenized)
            np.save(f"oof_prediction{fold_id}", oof_results.predictions)
            oof = df.query(f"fold == {fold_id}").copy()
            oof["pred0"] = oof_results.predictions[:, 0]
            oof["pred1"] = oof_results.predictions[:, 1]
            oof.to_csv(f"oof{fold_id}.csv", index=False)

            results = trainer.predict(test_dataset=test_tokenized)
            np.save(f"test_prediction{fold_id}", results.predictions)

loading file spiece.model from cache at /root/.cache/huggingface/hub/models--rinna--japanese-gpt2-medium/snapshots/f464b76739c884d8b0479a0a7705b7fa71c3fd5a/spiece.model
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--rinna--japanese-gpt2-medium/snapshots/f464b76739c884d8b0479a0a7705b7fa71c3fd5a/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--rinna--japanese-gpt2-medium/snapshots/f464b76739c884d8b0479a0a7705b7fa71c3fd5a/tokenizer_config.json


  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--rinna--japanese-gpt2-medium/snapshots/f464b76739c884d8b0479a0a7705b7fa71c3fd5a/config.json
Model config GPT2Config {
  "_name_or_path": "rinna/japanese-gpt2-medium",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 1,
  "embd_pdrop": 0.1,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1024,
  "n_head": 16,
  "n_inner": 4096,
  "n_layer": 24,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      

...

  0%|          | 0/2 [00:00<?, ?ba/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--rinna--japanese-gpt2-medium/snapshots/f464b76739c884d8b0479a0a7705b7fa71c3fd5a/config.json
Model config GPT2Config {
  "_name_or_path": "rinna/japanese-gpt2-medium",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 1,
  "embd_pdrop": 0.1,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1024,
  "n_head": 16,
  "n_inner": 4096,
  "n_layer": 24,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      

Step,Training Loss,Validation Loss,F1
250,No log,0.178557,0.231884
500,0.113800,0.079248,0.701754
750,0.113800,0.109615,0.724138
1000,0.033900,0.103561,0.710744
1250,0.033900,0.153596,0.740741
1500,0.006200,0.169812,0.701754
1750,0.006200,0.196169,0.709677
2000,0.001000,0.219549,0.701754
2250,0.001000,0.222261,0.696429
2500,0.000300,0.219302,0.690265


The following columns in the evaluation set don't have a corresponding argument in `GPT2ForSequenceClassification.forward` and have been ignored: text, __index_level_0__. If text, __index_level_0__ are not expected by `GPT2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1051
  Batch size = 64
The following columns in the evaluation set don't have a corresponding argument in `GPT2ForSequenceClassification.forward` and have been ignored: text, __index_level_0__. If text, __index_level_0__ are not expected by `GPT2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1051
  Batch size = 64
The following columns in the evaluation set don't have a corresponding argument in `GPT2ForSequenceClassification.forward` and have been ignored: text, __index_level_0__. If text, __index_level_0__ are not expected by `GPT2ForSequenceClassification.forward`,  you ca

The following columns in the test set don't have a corresponding argument in `GPT2ForSequenceClassification.forward` and have been ignored: text. If text are not expected by `GPT2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 3223
  Batch size = 64
